Microsoft project phase 1

In [32]:
#import pandas for data cleaning and manipulation
import pandas as pd
#import numpy for numerical operations
import numpy as np
# import seaborn and matplotlib for data visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 

Data Understanding

In [2]:
! ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
%%script sqlite3 im.db --out tables
.tables
.quit

In [4]:
#displaying all the tables in the SQL database
print (tables)

directors      movie_akas     movie_ratings  principals   
known_for      movie_basics   persons        writers      



In [5]:
#connecting to the SQL database using sqlite3
import sqlite3
conn = sqlite3.connect ("im.db")
cur = conn.cursor


In [19]:
# extracting the dataset to represent genres from the SQL data base 
films = pd.read_sql ("SELECT* from movie_basics;", conn)
films.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [7]:
#SQL database to represent consumer preferences
preferences = pd.read_sql ("SELECT* from movie_ratings;", conn)
preferences

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [8]:
# SQL dataset to represent income
income = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
income.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [9]:
films.genres.unique()

array(['Action,Crime,Drama', 'Biography,Drama', 'Drama', ...,
       'Music,Musical,Reality-TV', 'Animation,Crime',
       'Adventure,History,War'], dtype=object)

In [10]:
#checking for null values
films.isnull().sum()

movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64

In [11]:
preferences.isnull().sum()

movie_id         0
averagerating    0
numvotes         0
dtype: int64

In [12]:
income.isnull().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

Data Cleaning

Dealing with missing data

In [13]:
#detecting missing values
films.isna().sum()

movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64

In [15]:
#replacing null values in the runtime_minutes column with the median value
#The median value is less likely to influence the distribution of the dataset
films = films ['runtime_minutes'].fillna(films['runtime_minutes'].median())
films

0         175.0
1         114.0
2         122.0
3          87.0
4          80.0
          ...  
146139    123.0
146140     87.0
146141     87.0
146142    116.0
146143     87.0
Name: runtime_minutes, Length: 146144, dtype: float64

In [18]:
# genre column has categorical variables missing values are replaced with 'unknown' string
films = films['genres'].fillna("Unknown")
films

0           Action,Crime,Drama
1              Biography,Drama
2                        Drama
3                 Comedy,Drama
4         Comedy,Drama,Fantasy
                  ...         
146139                   Drama
146140             Documentary
146141                  Comedy
146142                 Unknown
146143             Documentary
Name: genres, Length: 146144, dtype: object

In [20]:
# original_title column  missing values are replaced with 'unknown' string
films = films['original_title'].fillna("Unknown")
films

0                                           Sunghursh
1                                     Ashad Ka Ek Din
2                          The Other Side of the Wind
3                                     Sabse Bada Sukh
4                               La Telenovela Errante
                             ...                     
146139                            Kuambil Lagi Hatiku
146140    Rodolpho Teóphilo - O Legado de um Pioneiro
146141                                Dankyavar Danka
146142                                         6 Gunn
146143                 Chico Albuquerque - Revelações
Name: original_title, Length: 146144, dtype: object

In [21]:
# ensuring the missing values have been eliminated from our dataset
films.isna()

0         False
1         False
2         False
3         False
4         False
          ...  
146139    False
146140    False
146141    False
146142    False
146143    False
Name: original_title, Length: 146144, dtype: bool

In [22]:
# checking for duplicates
films.duplicated().isna()

0         False
1         False
2         False
3         False
4         False
          ...  
146139    False
146140    False
146141    False
146142    False
146143    False
Name: original_title, Length: 146144, dtype: bool

Data Preparation

Joining tables

In [33]:
#obtaining the required columns from the SQL database 
combined = """
SELECT movie_basics.start_year, movie_basics.genres, movie_ratings.averagerating
FROM movie_basics
INNER JOIN movie_ratings
ON movie_basics.movie_id = movie_ratings.movie_id
;
"""
pd.read_sql (combined,conn)

,start_year,genres,averagerating
0,2013,"Action,Crime,Drama",7.0
1,2019,"Biography,Drama",7.2
2,2018,Drama,6.9
3,2018,"Comedy,Drama",6.1
4,2017,"Comedy,Drama,Fantasy",6.5
...,...,...,...
73851,2019,Documentary,6.2
73852,2019,"Drama,Family",8.7
73853,2017,Documentary,8.5
73854,2019,None,6.6


Data Analysis